In [1]:
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_doc2query.git
!pip install python-terrier==0.9.2
!pip install semanticscholar

  Cloning https://github.com/terrierteam/pyterrier_doc2query.git to c:\users\timm_\appdata\local\temp\pip-req-build-za3elqw_
  Resolved https://github.com/terrierteam/pyterrier_doc2query.git to commit 3449bcaf0727721d8d77c2524f0dc77dbbb77da0
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/terrierteam/pyterrier_doc2query.git 'C:\Users\timm_\AppData\Local\Temp\pip-req-build-za3elqw_'


# Vorgehen
1. Anfrage an das System senden
    1.1. Generate Querries from CLEF-IP and ACL
2. Bekommen Top K Dokumente
3. Co-Autoren Analyse auf den Dokumenten
4. Die Dokumente mit relevanten Dokumenten von Co-Autoren erweitern
5. Gruppen aus den Dokumenten (3 Gruppen a 6, 3, 1)
6. Größte Gruppe auswählen 
7. Termset C, mit den Termen die Expansion machen (Ansatz: Titel von Referenzen anstatt Abstracts verwenden)
8. B01 Gewichtung zur Termordnung (Alternative: tfidf, bm25)
9. B01 Unterart von DFR (Divergence from Randomness) ist bei Pyterrier zu finden
10. Nicht entscheidend B01 zu Nutzen, sondern eine Art von DFR
11. Semantic Scholar für Referenzen für jeweilige Publikation (haben auch ne API, Anfrage senden und Referenz zurück bekommen).


# Links
## ACL Anthology Corpus +  CLEF- IP
https://github.com/shauryr/ACL-anthology-corpus
http://www.ifs.tuwien.ac.at/~clef-ip/download-central.shtml

# 1. + 2. Anfrage an das System senden

In [ ]:
import pyterrier as pt
from semanticscholar import SemanticScholar
import os

if not pt.started():
  pt.init()

dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = 'F:\Bibliotheken\Desktop\Skripte\packgaabwir2022\indices\cord19'
pt_properties_path =os.path.join(pt_index_path + "/data.properties")


if not os.path.exists(pt_properties_path):
  indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)
  # Using SMART Stopword ATTENTION This is a global setting.
  indexer.setProperty("stopwords.filename", "F:\Bibliotheken\Desktop\Skripte\packgaabwir2022\en.txt")
  index_ref = indexer.index(dataset.get_corpus_iter(), 
                            fields=['title', 'doi', 'abstract'], 
                            meta=('docno',))
  
else:
  index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
  
index = pt.IndexFactory.of(index_ref)

In [ ]:
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
tf_idf.search("corona virus symptoms")

# 1.1. Generate Querries from CLEF-IP and ACL

In [ ]:
# Trying to generate querries from the dataset. Currently not working
# Try it with CLEF and ACL

import pyterrier_doc2query


if not pt.started():
    pt.init()

doc2query = pyterrier_doc2query.Doc2Query(append=True)
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = 'F:\Bibliotheken\Desktop\Skripte\packgaabwir2022\indices\cord19'
pt_properties_path = os.path.join(pt_index_path + "/data.properties")


if not os.path.exists(pt_properties_path):
    indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)
    indxer_pipe = doc2query >> indexer
    indxer_pipe.index(dataset.get_corpus_iter())
    indxer_pipe.setProperty(
        "stopwords.filename", "F:\Bibliotheken\Desktop\Skripte\packgaabwir2022\en.txt")
    index_ref = indxer_pipe.index(dataset.get_corpus_iter(),
                                  fields=['title', 'doi', 'abstract'],
                                  meta=('docno',))

else:
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

index = pt.IndexFactory.of(index_ref)

# 3. Co-Autoren Analyse auf den Dokumenten

Next steps:
Cycle through retrived docs. Get 

In [ ]:
import pandas as pd
metadata = pd.read_csv('~/.ir_datasets/cord19/2020-07-16/metadata.csv', low_memory=False)
metadata.columns

In [ ]:
index_count = 0


def prepare_index_path(indexName):
    global index_count
    index_count = index_count + 1
    index_path = indexName + str(index_count)

    if os.path.exists(index_path) & os.path.isdir(index_path):
        files = os.listdir(index_path)
        for file in files:
            file_name = index_path + '/' + file
            os.remove(file_name)
        os.rmdir(index_path)
    elif os.path.exists(index_path) & (not os.path.isdir(index_path)):
        os.rmove(index_path)

    return index_path


def build_index(docs):
    index_path_to_use = prepare_index_path("RELEVANCEIDX")
    indexer = pt.IterDictIndexer(
        './' + index_path_to_use, overwrite=True, blocks=True)
    indexer.setProperty(
        "stopwords.filename", "F:\Bibliotheken\Desktop\Skripte\packgaabwir2022\en.txt")
    index_created = indexer.index(docs["text"], docs["docno"], fields=['title', 'doi', 'abstract'],
                                  meta=('docno'))
    return index_created


In [ ]:

sch = SemanticScholar()

author_dict = {}
for id, authors in zip(metadata['cord_uid'], metadata['authors']):
    author_dict[id] = authors

# Add authors to retrieved docs
# Go through all docs split the authors by name
all_authors = doc['authors'].split('; ')
for author in all_authors:
    single_authors = author.split(', ')
    for single_author in single_authors:
        # ATTENTION Might need sleep(10) or sth because of API restrictions
        results = sch.search_author(single_author)
        if results:
            # Relevance? How to asses the relevance?
            # Build a new index with all the papers and assess relevance again
            indexref_author_papers = build_index(results[0].papers)
            ranked_author_papers = pt.BatchRetrieve(
                indexref_author_papers, wmodel='BM25')


An die original Title-Queries kommt ihr über Pyterrier mit:

    dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
    title_queries = dataset.get_topics('title')

title_queries ist ein Pandas Dataframe. Ich würde vorschlagen, dass ihr am besten die erweiterten Anfragen in das gleiche Dataframe-Format wie "title_queries" überführt ("expanded_title_queries" unten) und dann später dem BatchRetriever folgendermaßen übergebt.

    bm25 = pt.BatchRetrieve(index_ref , wmodel='BM25', num_results=10)
    res = bm25.transform(expanded_title_queries)
